# RNA-seq quantification

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1YN4FEJdWyKzPQziyFFCE7iyipXsVERwp?usp=drive_link

In [ ]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/sacCer3/bigZips/genes/sacCer3.ensGene.gtf.gz -O data/sacCer3.ensGene.gtf.gz

In [2]:
!featureCounts \
-p \
-a data/sacCer3.ensGene.gtf \
-o data/sacCer.featureCounts.tsv \
bam/WT_C_1.bam \
bam/WT_C_2.bam \
bam/WT_C_3.bam \
bam/WT_E_1.bam \
bam/WT_E_2.bam \
bam/WT_E_3.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.8

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 6 BAM files                                      ||
||                                                                            ||
||                           WT_C_1.bam                                       ||
||                           WT_C_2.bam                                       ||
||                           WT_C_3.bam                          

# Convert to count matrix
This command is essentially creating a simplified count matrix by:
1. Extracting only necessary columns (gene IDs and counts)
2. Cleaning up sample names
3. Reformatting to a tab-delimited file
4. Making it suitable for downstream analysis in R/DESeq2

Details explanation
1. `cat data/sacCer.featureCounts.tsv`
- Reads the content of the featureCounts output file
- featureCounts typically outputs a tab-separated file with gene counts

2. `awk '(NR>1) {printf "%s ", $1; for (i=7; i<=NF; i++) printf "%s ", $i; print ""}'`
- `NR>1`: Skips the header line
- `printf "%s ", $1`: Prints the first column (gene IDs)
- `for (i=7; i<=NF; i++)`: Loops through columns starting from 7th to last column
- These columns contain the actual count values
- First 6 columns in featureCounts output typically contain gene information (ID, Chr, Start, End, Strand, Length)

3. `sed s/bam//g`
- Removes 'bam' from sample names
- Cleans up file names

4. `tr -d "/"` 
- Removes forward slashes from the output
- Further cleans file paths

5. `tr -d "."` 
- Removes dots from the output
- Additional filename cleaning

6. `tr " " "\t"`
- Converts spaces to tabs
- Makes the output tab-delimited

7. `> data/sacCer_counts_raw.tsv`
- Saves the processed output to a new file
- Creates a clean count matrix with just gene IDs and counts

Example transformation:
```
Original featureCounts output:
GeneID  Chr  Start  End  Strand  Length  ./sample1.bam  ./sample2.bam
YDL248W  chr4  1802  2953  +  1152  45  67

After processing:
YDL248W  45  67
```

In [3]:
!cat data/sacCer.featureCounts.tsv | awk '(NR>1) {printf "%s ", $1; for (i=7; i<=NF; i++) printf "%s ", $i; print ""}' | sed s/bam//g | tr -d "/" | tr -d "." | tr " " "\t" > data/sacCer_counts_raw.tsv

In [5]:
import pandas as pd

df_gene_count = pd.read_csv("data/sacCer_counts_raw.tsv", sep="\t")
print(df_gene_count.shape)
df_gene_count.head()

(7127, 8)


,Geneid,WT_C_1,WT_C_2,WT_C_3,WT_E_1,WT_E_2,WT_E_3,Unnamed: 7
0,YDL248W,248,224,319,523,485,586,NaN
1,YDL247W-A,0,2,12,0,0,12,NaN
2,YDL247W,2,0,1,0,0,5,NaN
3,YDL246C,0,0,4,0,0,12,NaN
4,YDL245C,22,6,10,60,34,28,NaN
